In [5]:
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit
from pyspark.sql.functions import pandas_udf, PandasUDFType

import pandas as pd
from scipy import stats
import statsmodels.api as sm

import pyarrow as pa

In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("best_one").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.sql.execution.arrow.enabled", "true").\
        getOrCreate()

# Сделаем свободный DF для примера

In [14]:
df = spark.range(0, 10 * 1000).withColumn('id', (col('id') / 10000).cast('integer')).withColumn('v', rand())
df.cache()
df.count()

df.show()

+---+--------------------+
| id|                   v|
+---+--------------------+
|  0| 0.49840163843008656|
|  0|   0.069767152987945|
|  0|  0.7190715962156708|
|  0| 0.25640487040576976|
|  0|  0.6416189629727991|
|  0|  0.8528413886856091|
|  0|  0.6129146060898196|
|  0|  0.6164571002939396|
|  0|  0.6396182646364857|
|  0|  0.5752525667561637|
|  0|  0.8052223979650148|
|  0|  0.0405975490205146|
|  0|0.028272580413083004|
|  0|   0.594569306086507|
|  0|  0.9881616666059028|
|  0|  0.9770040363019834|
|  0|  0.9076460273114173|
|  0|  0.7452265863240174|
|  0| 0.12209934414804924|
|  0|  0.5801865543903545|
+---+--------------------+
only showing top 20 rows



# PANDAS UDF

In [15]:
# создадим первую функцию

    
@pandas_udf('double')    # декоратор
def plus_one(v):
    # базовая функция на python 
    return v + 1

df.withColumn('v', plus_one(df.v)).agg(count(col('v'))).show()

PythonException: 
  An exception was thrown from Python worker in the executor. The below is the Python worker stacktrace.
Traceback (most recent call last):
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 589, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 447, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 269, in read_single_udf
    return arg_offsets, wrap_scalar_pandas_udf(func, return_type)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 94, in wrap_scalar_pandas_udf
    arrow_return_type = to_arrow_type(return_type)
  File "/usr/bin/spark-3.0.0-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/sql/pandas/types.py", line

In [4]:
@pandas_udf("double", PandasUDFType.SCALAR)  # декоратор + тип данных
def pandas_plus_one(v):
    return v + 1

%timeit df.withColumn('v', pandas_plus_one(df.v)).agg(count(col('v'))).show()

+--------+
count(v)|
+--------+
10000000|
+--------+

+--------+
count(v)|
+--------+
10000000|
+--------+

+--------+
count(v)|
+--------+
10000000|
+--------+

+--------+
count(v)|
+--------+
10000000|
+--------+

1 loops, best of 3: 1.07 s per loop

In [5]:
# итеграция других библиотек (scipy)

@udf('double')
def cdf(v):
    return float(stats.norm.cdf(v))

%timeit df.withColumn('cumulative_probability', cdf(df.v)).agg(count(col('cumulative_probability'))).show()

+-----------------------------+
count(cumulative_probability)|
+-----------------------------+
 10000000|
+-----------------------------+

+-----------------------------+
count(cumulative_probability)|
+-----------------------------+
 10000000|
+-----------------------------+

+-----------------------------+
count(cumulative_probability)|
+-----------------------------+
 10000000|
+-----------------------------+

+-----------------------------+
count(cumulative_probability)|
+-----------------------------+
 10000000|
+-----------------------------+

1 loops, best of 3: 4min 13s per loop

In [6]:
@pandas_udf('double', PandasUDFType.SCALAR)
def pandas_cdf(v):
    return pd.Series(stats.norm.cdf(v))

%timeit df.withColumn('cumulative_probability', pandas_cdf(df.v)).agg(count(col('cumulative_probability'))).show()

+-----------------------------+
count(cumulative_probability)|
+-----------------------------+
 10000000|
+-----------------------------+

+-----------------------------+
count(cumulative_probability)|
+-----------------------------+
 10000000|
+-----------------------------+

+-----------------------------+
count(cumulative_probability)|
+-----------------------------+
 10000000|
+-----------------------------+

+-----------------------------+
count(cumulative_probability)|
+-----------------------------+
 10000000|
+-----------------------------+

1 loops, best of 3: 1.24 s per loop

In [7]:
# передача более сложных объектов и возврат нового объекта
@udf(ArrayType(df.schema))
def substract_mean(rows):
    vs = pd.Series([r.v for r in rows])
    vs = vs - vs.mean()
    return [Row(id=rows[i]['id'], v=float(vs[i])) for i in range(len(rows))]
  
%timeit df.groupby('id').agg(collect_list(struct(df['id'], df['v'])).alias('rows')).withColumn('new_rows', substract_mean(col('rows'))).withColumn('new_row', explode(col('new_rows'))).withColumn('id', col('new_row.id')).withColumn('v', col('new_row.v')).agg(count(col('v'))).show()

+--------+
count(v)|
+--------+
10000000|
+--------+

+--------+
count(v)|
+--------+
10000000|
+--------+

+--------+
count(v)|
+--------+
10000000|
+--------+

+--------+
count(v)|
+--------+
10000000|
+--------+

1 loops, best of 3: 2min 4s per loop

In [8]:
# передача более сложных объектов и возврат нового DF (+ новая колонка)

@pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)
# Input/output == pandas.DataFrame
def pandas_subtract_mean(pdf):
	return pdf.assign(v=pdf.v - pdf.v.mean())

%timeit df.groupby('id').apply(pandas_subtract_mean).agg(count(col('v'))).show()

+--------+
count(v)|
+--------+
10000000|
+--------+

+--------+
count(v)|
+--------+
10000000|
+--------+

+--------+
count(v)|
+--------+
10000000|
+--------+

+--------+
count(v)|
+--------+
10000000|
+--------+

1 loops, best of 3: 4.65 s per loop

In [9]:
df2 = df.withColumn('y', rand()).withColumn('x1', rand()).withColumn('x2', rand()).select('id', 'y', 'x1', 'x2')
df2.show()                                                               

+---+-------------------+--------------------+-------------------+
 id| y| x1| x2|
+---+-------------------+--------------------+-------------------+
 0| 0.681296060112836| 0.1288311079969241| 0.8181088445104816|
 0| 0.8888773955549102| 0.25520608131769806| 0.8371196660049978|
 0|0.12865336389189674| 0.05949353129319879|0.31240880781369607|
 0|0.33102706063173315| 0.3184970944017924| 0.9934788617057889|
 0|0.08530551734633984| 0.849098348411309|0.25958206625946156|
 0| 0.1142436882234027| 0.09221618780441287|0.06660852847156451|
 0| 0.3734801477601759| 0.16175735111155454|0.23741551784520665|
 0| 0.4626832884602221| 0.4090520759820342|0.21143786407406573|
 0| 0.3089074870133427| 0.7875508394004922|0.20291186344825263|
 0| 0.6963359144225203| 0.24446551311290765|0.09530396721263867|
 0|0.18601574521309183| 0.18283878773443607| 0.9049305072841698|
 0| 0.9986921036424282| 0.5744991393169917| 0.4377204256577574|
 0|0.47413548244645665| 0.8647990390377169| 0.6145253333423468|
 0| 0.8678090740409161| 0.9349286905893688| 0.897022900084491|
 0| 0.6752577347437083| 0.20625908730646103|0.10315736062362346|
 0|0.22669523505013633| 0.6099324032866738| 0.8357508819755833|
 0| 0.6880907870618188| 0.8392228400945341| 0.7226505258273653|
 0|0.30101130104653884| 0.9651274666079585| 0.5422836657606281|
 0| 0.7195503022011948| 0.9288544640693567|0.03643847265357025|
 0|0.27410622722360234|0.051428600469085706| 0.7646588630569261|
+---+-------------------+--------------------+-------------------+
only showing top 20 rows

## UDF и агрегация данных

In [10]:
group_column = 'id'
y_column = 'y'
x_columns = ['x1', 'x2']
# записываем отдельно схему колонок
schema = df2.select(group_column, *x_columns).schema

#передаем схему и тип объекта, который будет приходить на вход
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
# Input/output == pandas.DataFrame
def ols(pdf):
    group_key = pdf[group_column].iloc[0]
    y = pdf[y_column]
    X = pdf[x_columns]
    X = sm.add_constant(X)
    
    # применяем метод из библиотеки
    model = sm.OLS(y, X).fit()
    
    # создаем новый DF
    return pd.DataFrame([[group_key] + [model.params[i] for i in   x_columns]], columns=[group_column] + x_columns)

beta = df2.groupby(group_column).apply(ols)
beta.show()

+---+--------------------+--------------------+
 id| x1| x2|
+---+--------------------+--------------------+
148|-0.00917654654646...|-0.01005665429668...|
463|-0.01500674870033...|0.016025010035176222|
471|-0.00832321162854...|-0.00404915919899...|
496|-0.01122055554723648|-0.00280962655140...|
833|-0.01629553352114...|0.002779410391406...|
243|0.005259499685366535|0.011281848381048665|
392|0.005589240115972826|-0.00950385069041...|
540|5.918574070326934...|0.012159354453070217|
623|0.020442434869455878|0.004083702101312427|
737|0.006226657113389954|0.003961770851249408|
858|0.001940560121997...|0.006720865070135...|
897|-0.00142535705654...|0.004045227546180374|
 31|0.005465606169062085|0.008832031597331093|
516|-0.00531332000715...|0.001981946321763...|
 85|0.012725673978444558|-0.02828045053679735|
137|-0.00131062800808...|-5.30640018178707...|
251|0.006229489454833485|0.002962616001996...|
451|0.003804104279762211|-0.00447206880074...|
580|0.026962287867315624|3.293459638984281E-4|
808|-0.01025147566168...|0.004950671582079154|
+---+--------------------+--------------------+
only showing top 20 rows